In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [4]:
pip install tabulate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import os
os.listdir()

['.config', 'dataset_final_all.csv', 'sample_data']

In [6]:
dt=pd.read_csv("dataset_final_all.csv")
dt.tail()

,Unnamed: 0,stars,review_body,review_title,niveau,language,target,review_all
11995,60925,2,I am disappointed with the Kona Grill Mat. It ...,Best Grill Mat???,insatisfait,en,-1,Best Grill Mat??? I am disappointed with the K...
11996,45700,2,Lasted two months before strap holder broke - ...,Lasted Two Months,insatisfait,en,-1,Lasted Two Months Lasted two months before str...
11997,69161,2,"Disappointed in these. The tips don't line up,...",Poor quality,insatisfait,en,-1,Poor quality Disappointed in these. The tips d...
11998,71355,2,Battery charger lasted a day Poor construction,Two Stars,insatisfait,en,-1,Two Stars Battery charger lasted a day Poor co...
11999,59534,2,Product worked for 4 months then stopped. Look...,Product did not work,insatisfait,en,-1,Product did not work Product worked for 4 mont...


In [7]:
def assign_label(val):
  if val=='satisfait':
    return 2
  elif val=='mitige':
    return 1
  else:
    return 0

In [8]:
dt['target']=dt['niveau'].apply(lambda x:assign_label(x))
dt

,Unnamed: 0,stars,review_body,review_title,niveau,language,target,review_all
0,143119,4,La house est conforme à l'offre. Elle à été bi...,Bon produit,satisfait,fr,2,Bon produit La house est conforme à l'offre. E...
1,145657,4,Pour faire court le produit était mal fermé ma...,Très bon shampooing pour enlever les reflets j...,satisfait,fr,2,Très bon shampooing pour enlever les reflets j...
2,157590,4,bon produit qualité = au prix,utilisation facile,satisfait,fr,2,utilisation facile bon produit qualité = au prix
3,151657,4,Super rapport qualité/prix. Ce sont des pièces...,Satisfait,satisfait,fr,2,Satisfait Super rapport qualité/prix. Ce sont ...
4,124029,4,"Le colis a mis le temps pour venir, mais il es...",Maillot de sport 2 étoiles,satisfait,fr,2,Maillot de sport 2 étoiles Le colis a mis le t...
...,...,...,...,...,...,...,...,...
11995,60925,2,I am disappointed with the Kona Grill Mat. It ...,Best Grill Mat???,insatisfait,en,0,Best Grill Mat??? I am disappointed with the K...
11996,45700,2,Lasted two months before strap holder broke - ...,Lasted Two Months,insatisfait,en,0,Lasted Two Months Lasted two months before str...
11997,69161,2,"Disappointed in these. The tips don't line up,...",Poor quality,insatisfait,en,0,Poor quality Disappointed in these. The tips d...
11998,71355,2,Battery charger lasted a day Poor construction,Two Stars,insatisfait,en,0,Two Stars Battery charger lasted a day Poor co...


In [9]:
text = dt.review_all.values 
label = dt.target.values

In [10]:
tokenizer = BertTokenizer.from_pretrained( 
    'bert-base-multilingual-cased', 
    do_lower_case = True 
    )

In [11]:
def print_rand_sentence():
  '''Displays the tokens and respective IDs of a random text sample'''
  index = random.randint(0, len(text)-1)
  table = np.array([tokenizer.tokenize(text[index]), 
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
  print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

╒══════════╤═════════════╕
│ Tokens   │   Token IDs │
╞══════════╪═════════════╡
│ coli     │       77019 │
├──────────┼─────────────┤
│ ##s      │       10107 │
├──────────┼─────────────┤
│ vol      │       12714 │
├──────────┼─────────────┤
│ ##e      │       10112 │
├──────────┼─────────────┤
│ le       │       10141 │
├──────────┼─────────────┤
│ vende    │       64298 │
├──────────┼─────────────┤
│ ##ur     │       10546 │
├──────────┼─────────────┤
│ a        │         169 │
├──────────┼─────────────┤
│ de       │       10104 │
├──────────┼─────────────┤
│ ##pose   │       50864 │
├──────────┼─────────────┤
│ le       │       10141 │
├──────────┼─────────────┤
│ coli     │       77019 │
├──────────┼─────────────┤
│ ##s      │       10107 │
├──────────┼─────────────┤
│ dans     │       10260 │
├──────────┼─────────────┤
│ la       │       10109 │
├──────────┼─────────────┤
│ bo       │       20506 │
├──────────┼─────────────┤
│ ##ite    │       12704 │
├──────────┼─────────────┤
│

In [12]:

token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 24,
                        truncation=True,
                        padding= 'max_length',
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
label = torch.tensor(label)

In [13]:
token_id[10]

tensor([  101, 67695, 29264, 11681, 67695,   106, 22242, 12150, 10446, 65937,
        10112,   117, 10144,   180, 72894, 34508, 13477, 10152, 17916,   119,
        20445, 19685, 11481,   102])

In [15]:
def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table, 
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence_encoding()

╒══════════╤═════════════╤══════════════════╕
│ Tokens   │   Token IDs │   Attention Mask │
╞══════════╪═════════════╪══════════════════╡
│ [CLS]    │         101 │                1 │
├──────────┼─────────────┼──────────────────┤
│ bien     │       12028 │                1 │
├──────────┼─────────────┼──────────────────┤
│ mais     │       10614 │                1 │
├──────────┼─────────────┼──────────────────┤
│ .        │         119 │                1 │
├──────────┼─────────────┼──────────────────┤
│ .        │         119 │                1 │
├──────────┼─────────────┼──────────────────┤
│ .        │         119 │                1 │
├──────────┼─────────────┼──────────────────┤
│ aim      │       56048 │                1 │
├──────────┼─────────────┼──────────────────┤
│ ##ant    │       11236 │                1 │
├──────────┼─────────────┼──────────────────┤
│ un       │       10119 │                1 │
├──────────┼─────────────┼──────────────────┤
│ peu      │       14574 │        

## Fractionnement des données

In [16]:

val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 8

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(label)),
    test_size = val_ratio,
    shuffle = True,
    stratify = label)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          label[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        label[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [17]:

def b_tp(preds, label):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == label and preds == 1 for preds, label in zip(preds, label)])

def b_fp(preds, label):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != label and preds == 1 for preds, label in zip(preds, label)])

def b_tn(preds, label):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == label and preds == 0 for preds, label in zip(preds, label)])

def b_fn(preds, label):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != label and preds == 0 for preds, label in zip(preds, label)])

def b_kn(preds, label):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == label and preds == -1 for preds, label in zip(preds, label)])

def b_kp(preds, label):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != label and preds == -1 for preds, label in zip(preds, label)])

def b_metrics(preds, label):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, label)
  tn = b_tn(preds, label)
  fp = b_fp(preds, label)
  fn = b_fn(preds, label)
  b_accuracy = (tp + tn) / len(label)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity


In [18]:


def b_tp(preds, label):
    '''Returns True Positives (TP): count of correct predictions of actual class 1'''
    return sum([preds == label and preds == 1 for preds, label in zip(preds, label)])

def b_fp(preds, label):
    '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
    return sum([preds != label and preds == 1 for preds, label in zip(preds, label)])

def b_tn(preds, label):
    '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
    return sum([preds == label and preds == 0 for preds, label in zip(preds, label)])

def b_fn(preds, label):
    '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
    return sum([preds != label and preds == 0 for preds, label in zip(preds, label)])

def b_kn(preds, label):
    '''Returns True Negatives (TN): count of correct predictions of actual class -1'''
    return sum([preds == label and preds == -1 for preds, label in zip(preds, label)])

def b_kp(preds, label):
    '''Returns False Negatives (FN): count of wrong predictions of actual class -1'''
    return sum([preds != label and preds == -1 for preds, label in zip(preds, label)])

def b_metrics(preds, label):
    '''
    Returns the following metrics:
        - accuracy    = (TP + TN) / N
        - precision   = TP / (TP + FP)
        - recall      = TP / (TP + FN)
        - specificity = TN / (TN + FP)
    '''
    preds = np.argmax(preds, axis=1).flatten()
    label = label.flatten()
    tp = b_tp(preds, label)
    tn = b_tn(preds, label)
    fp = b_fp(preds, label)
    fn = b_fn(preds, label)
    kn = b_kn(preds, label)
    kp = b_kp(preds, label)
    b_accuracy = (tp + tn + kn) / len(label)
    b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
    b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
    b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
    return b_accuracy, b_precision, b_recall, b_specificity


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
os.environ['TORCH_USE_CUDA_DSA'] = "1"
import torch

In [ ]:
os.environ['TORCH_USE_CUDA_DSA'] = "1"
import torch


In [20]:
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]


RuntimeError: ignored